# Notebook 01: Data Preparation & Audit

## Table of Contents
1. [Setup and Data Ingestion](#setup-and-data-ingestion)
    * Load Raw Data
    * Structural Audit (Shape, Dtypes)
2. [Data Sanitation](#data-sanitation)
    * Missing Value Check
    * Duplicate Removal
3. [Temporal Alignment](#temporal-alignment)
    * UTC Standardization
    * Date Normalization
4. [Data Integration](#data-integration)
    * Sentiment Merge
    * Neutral Fill Strategy
5. [Outlier Management](#outlier-management)
    * Leverage & PnL Extreme Value Detection
    * Capping Policy Application
6. [Engineering Reasoning Log](#engineering-reasoning-log)
7. [Export](#export)

### 1. Setup and Data Ingestion <a id="setup-and-data-ingestion"></a>

In [7]:
import pandas as pd
import numpy as np
import os

RAW_DATA_PATH = '../data'
PROCESSED_DATA_PATH = '../data/processed'
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

trader_df = pd.read_csv(os.path.join(RAW_DATA_PATH, 'historical_data.csv'))
sentiment_df = pd.read_csv(os.path.join(RAW_DATA_PATH, 'fear_greed_index.csv'))

print(f"Trader Data Shape: {trader_df.shape}")
print(f"Trader Data Columns: {trader_df.columns.tolist()}")
print(f"Trader Data Types:\n{trader_df.dtypes}")
print("-" * 30)
print(f"Sentiment Data Shape: {sentiment_df.shape}")

Trader Data Shape: (211224, 16)
Trader Data Columns: ['Account', 'Coin', 'Execution Price', 'Size Tokens', 'Size USD', 'Side', 'Timestamp IST', 'Start Position', 'Direction', 'Closed PnL', 'Transaction Hash', 'Order ID', 'Crossed', 'Fee', 'Trade ID', 'Timestamp']
Trader Data Types:
Account                 str
Coin                    str
Execution Price     float64
Size Tokens         float64
Size USD            float64
Side                    str
Timestamp IST           str
Start Position      float64
Direction               str
Closed PnL          float64
Transaction Hash        str
Order ID              int64
Crossed                bool
Fee                 float64
Trade ID            float64
Timestamp           float64
dtype: object
------------------------------
Sentiment Data Shape: (2644, 4)


### 2. Data Sanitation <a id="data-sanitation"></a>

In [8]:
null_counts = trader_df.isnull().sum()
print(f"Null Values:\n{null_counts[null_counts > 0]}")

dup_count = trader_df.duplicated().sum()
print(f"Duplicate Rows Detected: {dup_count}")

trader_df = trader_df.drop_duplicates()

Null Values:
Series([], dtype: int64)
Duplicate Rows Detected: 0


### 3. Temporal Alignment <a id="temporal-alignment"></a>

In [9]:
trader_df['datetime_utc'] = pd.to_datetime(trader_df['Timestamp'], unit='ms', utc=True)
trader_df['date'] = trader_df['datetime_utc'].dt.normalize()

sentiment_df['date'] = pd.to_datetime(sentiment_df['date'], utc=True)
sentiment_clean = sentiment_df[['date', 'value', 'classification']].rename(
    columns={'value': 'sentiment_score', 'classification': 'sentiment_class'}
)

### 4. Data Integration <a id="data-integration"></a>

In [10]:
merged_df = pd.merge(
    left=trader_df,
    right=sentiment_clean,
    on='date',
    how='left'
)

merged_df['sentiment_score'] = merged_df['sentiment_score'].fillna(50)
merged_df['sentiment_class'] = merged_df['sentiment_class'].fillna('Neutral')

### 5. Outlier Management <a id="outlier-management"></a>

In [11]:
def cap_outliers(series, lower_quantile=0.01, upper_quantile=0.99):
    lower_limit = series.quantile(lower_quantile)
    upper_limit = series.quantile(upper_quantile)
    return series.clip(lower=lower_limit, upper=upper_limit)

merged_df['leverage_capped'] = cap_outliers(merged_df['Size USD'] / merged_df['Start Position'].replace(0, np.nan))
merged_df['pnl_capped'] = cap_outliers(merged_df['Closed PnL'])

print(merged_df[['Closed PnL', 'pnl_capped']].describe())

          Closed PnL     pnl_capped
count  211224.000000  211224.000000
mean       48.749001      31.306126
std       919.164828     134.809828
min   -117990.104100    -140.766890
25%         0.000000       0.000000
50%         0.000000       0.000000
75%         5.792797       5.792797
max    135329.090100    1023.315314


###  6. Engineering Reasoning Log
<a id="engineering-reasoning-log"></a>

| **Phase** | **Action Taken** | **Engineering Justification** |
| --- | --- | --- |
| **Ingestion** | Loaded raw CSVs and audited types. | Ensures schema consistency and prevents downstream failures caused by silent type mismatches (e.g., strings in float columns). |
| **Sanitation** | Removed full-row duplicates. | Eliminates redundant transaction logs that artificially inflate trade frequency and volume metrics. |
| **Alignment** | Converted Unix MS to UTC Datetime. | Standardizes diverse timezones into a single global reference frame to ensure accurate event sequencing. |
| **Integration** | Left Join on UTC Date; Filled Missing Sentiment with 50 (Neutral). | Preserves all trade data while handling data gaps conservatively; assumes market neutrality when sentiment signal is absent. |
| **Outlier Logic** | Applied 1st/99th Percentile Clipping to PnL and Leverage. | Mitigates the impact of "fat-finger" errors or liquidation wicks that would otherwise skew mean/variance calculations in the model. |

### 7.Export <a id="export"></a>

In [13]:
output_path = os.path.join(PROCESSED_DATA_PATH, '01_merged_data.csv')
merged_df.to_csv(output_path, index=False)
print(f"Data saved to: {output_path}")

Data saved to: ../data/processed\01_merged_data.csv
